# RAG Korištenjem Azure AI Agent Service i Semantic Kernel

Ovaj isječak koda pokazuje kako stvoriti i upravljati Azure AI agentom za generiranje uz prošireno dohvaćanje (RAG) koristeći `Azure AI Agent Service` i `Semantic Kernel`. Agent obrađuje korisničke upite na temelju dohvaćenog konteksta i pruža točne odgovore u skladu s tim.


Popravak verzije SQLite  
Ako naiđete na pogrešku:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Otkomentirajte ovaj blok koda na početku svoje bilježnice:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Uvoz paketa
Sljedeći kod uvozi potrebne pakete:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Generacija uz pomoć pretraživanja s Semantic Kernelom i Azure AI Agent Serviceom

Ovaj primjer pokazuje kako koristiti **Azure AI Agent Service** za izvođenje **Generacije uz pomoć pretraživanja (RAG)** kombiniranjem jezičnog modela s kontekstom specifičnim za domenu iz prenesenog dokumenta.

### Kako Funkcionira

1. **Prenos dokumenta**: Markdown datoteka (document.md) koja sadrži informacije (Contosova politika putnog osiguranja) prenosi se u agent servis.

2. **Stvaranje vektorske pohrane**: Dokument se indeksira u vektorsku pohranu kako bi se omogućilo semantičko pretraživanje njegovog sadržaja.

3. **Konfiguracija agenta**: Agent se postavlja koristeći model `gpt-4o` s sljedećim strogim uputama:
   - Odgovarati samo na pitanja temeljem sadržaja pronađenog u dokumentu.
   - Odbiti odgovoriti ako je pitanje izvan opsega.

4. **Integracija alata za pretraživanje datoteka**: `FileSearchTool` se registrira s agentom, omogućujući modelu da tijekom inferencije pretražuje i dohvaća relevantne dijelove iz indeksiranog dokumenta.

5. **Interakcija s korisnikom**: Korisnici mogu postavljati pitanja. Ako se pronađu relevantne informacije u dokumentu, agent generira utemeljen odgovor.  
   Ako ne, agent eksplicitno odgovara da dokument ne sadrži dovoljno informacija.


### Glavna Funkcija

```python
# Ova funkcija je glavni ulaz u aplikaciju
def main():
    print("Dobrodošli u naš program!")  # Ispisuje poruku dobrodošlice
    user_input = input("Unesite svoje ime: ")  # Traži od korisnika da unese svoje ime
    print(f"Pozdrav, {user_input}!")  # Prikazuje personaliziranu poruku dobrodošlice
```

### Pomoćne Funkcije

```python
# Ova funkcija provjerava je li broj paran
def is_even(number):
    return number % 2 == 0  # Vraća True ako je broj paran, inače False

# Ova funkcija provjerava je li broj neparan
def is_odd(number):
    return number % 2 != 0  # Vraća True ako je broj neparan, inače False
```

### Upute za Korištenje

1. Pokrenite datoteku koristeći Python.
2. Slijedite upute na ekranu.
3. Unesite svoje ime kada se to od vas zatraži.
4. Koristite funkcije `is_even` i `is_odd` za provjeru parnosti brojeva.

[!NOTE] Ovaj program je namijenjen za edukativne svrhe.

### Primjeri Korištenja

```python
# Provjera parnosti broja
number = 4
if is_even(number):
    print(f"{number} je paran broj.")  # Ispisuje ako je broj paran
else:
    print(f"{number} je neparan broj.")  # Ispisuje ako je broj neparan
```

[!TIP] Isprobajte različite brojeve kako biste testirali funkcije.


Provjerite da ste prvo pokrenuli `az login` koristeći Azure CLI kako bi se osigurao odgovarajući kontekst autentifikacije prilikom korištenja `DefaultAzureCredential`. Azure AI Agent Service ne koristi API ključeve.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Naravno! Molim vas da mi dostavite sadržaj datoteke kako bih je mogao prevesti na hrvatski.



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije koje proizlaze iz korištenja ovog prijevoda.
